In [1]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
import matplotlib.pyplot as plt
import math as mt
import tensorflow as tf
fp_padrao = 0.92
fp_padrao_Sup = 2-fp_padrao

In [5]:
def normaliza(s):
    if s >= 0:
       return s
    elif s < 0 :
       return s + 2

base = pd.read_excel('testeexport.xlsx')
base1 = base

base.drop(columns=['Freqüência', 'Ocorrência'],inplace=True)

base.rename(columns ={'Fat.Pot T': 'FP T',
                      'Fat.Pot. R': 'FP R',
                      'Fat.Pot. S': 'FP S'}, inplace=True)

base = pd.pivot_table(base, index=['Data'])

base['Pot R KVA'] = (base['Corrente R'] * base['Tensão R'] * (mt.sqrt(3)))/1000
base['Pot S KVA'] = (base['Corrente S'] * base['Tensão S'] * (mt.sqrt(3)))/1000
base['Pot T KVA'] = (base['Corrente T'] * base['Tensão T'] * (mt.sqrt(3)))/1000

#Calculo KVAr usando numpy
base['Pot R KW'] = (base['Pot R KVA'] * (base['FP R']))
base['Pot S KW'] = (base['Pot S KVA'] * (base['FP S']))
base['Pot T KW'] = (base['Pot T KVA'] * (base['FP T']))

#Calculo KVAr usando numpy
base['Pot R KVAr'] = np.sqrt(np.power(base['Pot R KVA'],2) - np.power(base['Pot R KW'],2))
base['Pot S KVAr'] = np.sqrt(np.power(base['Pot S KVA'],2) - np.power(base['Pot S KW'],2))
base['Pot T KVAr'] = np.sqrt(np.power(base['Pot T KVA'],2) - np.power(base['Pot T KW'],2))

#Calculo Faltante Kvar
base['Pot R KVAr faltante'] = (base['Pot R KVAr']) - (np.sqrt(np.power(base['Pot R KVA'],2)-
                                                                      (np.power((base['Pot R KVA'] * (fp_padrao)),2))))
base['Pot S KVAr faltante'] = (base['Pot S KVAr']) - (np.sqrt(np.power(base['Pot S KVA'],2)-
                                                                      (np.power((base['Pot S KVA'] * (fp_padrao)),2))))
base['Pot T KVAr faltante'] = (base['Pot T KVAr']) - (np.sqrt(np.power(base['Pot T KVA'],2)-
                                                                      (np.power((base['Pot T KVA'] * (fp_padrao)),2))))

base["FP R"] = base["FP R"].apply(normaliza)
base["FP S"] = base["FP S"].apply(normaliza)
base["FP T"] = base["FP T"].apply(normaliza)

base["FP R"] = base["FP R"].apply(normaliza)
base["FP S"] = base["FP S"].apply(normaliza)
base["FP T"] = base["FP T"].apply(normaliza)

fpfases_abaixo = base.loc[((base["FP R"]< fp_padrao) | (base["FP R"]> fp_padrao_Sup))
                         | ((base["FP S"]< fp_padrao) | (base["FP S"]> fp_padrao_Sup))
                         | ((base["FP T"]< fp_padrao) | (base["FP T"]> fp_padrao_Sup))]

fpfases = base.loc[((base["FP R"]< fp_padrao) | (base["FP R"]> fp_padrao_Sup))
                         | ((base["FP S"]< fp_padrao) | (base["FP S"]> fp_padrao_Sup))
                         | ((base["FP T"]< fp_padrao) | (base["FP T"]> fp_padrao_Sup))]

maximoR = fpfases['Pot R KVAr faltante'].max()
maximoS = fpfases['Pot S KVAr faltante'].max()
maximoT = fpfases['Pot T KVAr faltante'].max()

In [7]:
def fases():
    exceldados = pd.ExcelWriter('FP Fases.xlsx',
                                engine='xlsxwriter',
                                datetime_format='mmm-d-yyyy hh:mm:ss',
                                date_format='mmmm-dd-yyyy',)
    base.round(5).to_excel(exceldados, sheet_name='Tiago') #1
    fpfases.round(5).to_excel(exceldados, sheet_name='FP Fora do padrão') #2
    exceldados.save()

    from win32com.client import Dispatch
    appexcel = Dispatch('Excel.Application')
    wb = appexcel.Workbooks.Open('C:\Dev\Energens\EnergensDev\Tiago - São Pedro do Butiá\FP Fases.xlsx')
    for i in range (1, 3, 1):
        appexcel.Worksheets(i).Activate()
        appexcel.ActiveSheet.Columns.AutoFit()
    appexcel.Worksheets(1).Activate()
    wb.Save()
    wb.Close()

def graficos():
    exceldados = pd.ExcelWriter('FP Fases gráficos.xlsx',
                                engine='xlsxwriter',
                                datetime_format='mmm-d-yyyy hh:mm:ss',
                                date_format='mmmm-dd-yyyy',)
    base.round(5).to_excel(exceldados, sheet_name='Tiago') #1
    fpfases.round(5).to_excel(exceldados, sheet_name='FP Fora do padrão') #2
    exceldados.save()

    from win32com.client import Dispatch
    appexcel = Dispatch('Excel.Application')
    wb = appexcel.Workbooks.Open('C:\Dev\Energens\EnergensDev\Tiago - São Pedro do Butiá\FP Fases gráficos.xlsx')
    for i in range (1, 3, 1):
        appexcel.Worksheets(i).Activate()
        appexcel.ActiveSheet.Columns.AutoFit()
    appexcel.Worksheets(1).Activate()
    wb.Save()
    wb.Close()

In [8]:
fases()